In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision as tv
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# 1. 加载数据集
train_df = pd.read_csv(r'../datasets/sgg-data/fashion-mnist_train.csv') # (60000, 785)
test_df = pd.read_csv(r'../datasets/sgg-data/fashion-mnist_test.csv') # (10000, 785)

# 2. 数据预处理
# 2.1 划分数据集
train_x = train_df.iloc[:, 1:] / 255.0 # (60000, 784)
train_y = train_df.iloc[:, 0] # (60000, )

# 2.2 转为tensor
tensor_train_x = torch.tensor(train_x.values, dtype=torch.float32)
tensor_train_y = torch.tensor(train_y.values, dtype=torch.int64)

In [3]:
# 把 784 --> 1 * 28 * 28 (c, h, w)
tensor_train_x = tensor_train_x.view(-1, 1, 28, 28) # (60000, 1, 28, 28)
print(tensor_train_x[0])

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0157, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.2431, 0.2392, 0.0824, 0.1137, 0.0902, 0.2000,
          0.5333, 0.2392, 0.0000, 0.0000, 0.0000, 0.0000,

In [4]:
# 准备包装为Dataset
dataset_train = TensorDataset(tensor_train_x, tensor_train_y)

# 定义一些超参数
batch_size, lr, epochs = 128, 0.5, 10

# 创建DataLoader
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

In [5]:
# 创建模型
class MyModel(nn.Module):
    def __init__(self, num_classes=10): # 输入的图像宽高为(28*28)
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, 3, 1) # 输入通道数1，输出通道数32，卷积核大小3，步长1
        self.conv2 = nn.Conv2d(64, 32, 3, 1)
        self.pool = nn.MaxPool2d(2, 2)
        # to do
        self.fc1 = nn.Linear(32 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        t1 = self.pool(torch.relu(self.conv1(x)))
        t2 = self.pool(torch.relu(self.conv2(t1)))
        t2 = t2.view(-1, 32 * 5 * 5)
        t3 = torch.relu(self.fc1(t2))
        y = self.fc2(t3)
        return y


In [8]:
model = MyModel()
print(type(model.parameters()))
for param in model.parameters():
    print(param.shape)

<class 'generator'>
torch.Size([64, 1, 3, 3])
torch.Size([64])
torch.Size([32, 64, 3, 3])
torch.Size([32])
torch.Size([128, 800])
torch.Size([128])
torch.Size([10, 128])
torch.Size([10])
